In [8]:
# !git clone https://github.com/akshitv2/VAE-latent-space-experiment.git
# %cd /content/VAE-latent-space-experiment

fatal: destination path 'VAE-latent-space-experiment' already exists and is not an empty directory.
/content/VAE-latent-space-experiment


In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os
from tqdm import tqdm

from experiments.Checkpointing import save_checkpoint
from models.VAE import VAE
from modules.Losses import vae_loss
from modules.SaveOutputs import save_reconstructions, save_samples

dataset_dir: str = "./data/raw"
out_dir: str = "./outputs/"
batch_size: int = 64
latent_dim: int = 512
checkpoint_dir = "./experiments/checkpoints"
epochs: int = 10
lr: float = 3e-4
beta: float = 0.5

torch.manual_seed(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Resize((64, 64)),  # resize to 224x224
    transforms.ToTensor()  # convert to tensor & scale to [0,1]
])


In [23]:
dataset = datasets.ImageFolder(root="G:\Temp", transform=transform)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)

In [10]:


print("Loaded datasets, number of samples: ", len(dataset))

# Model & Optimizer
model = VAE(latent_dim=latent_dim).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


global_step = 0
os.makedirs(out_dir, exist_ok=True)

Loaded datasets, number of samples:  202599


In [11]:
train_mode = True
current_epoch = 0
epochs = 2
beta = 0.001
beta_anneal_factor = 1.1

In [20]:
training_loss_tracker = []

In [29]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F

# -------------------------------
# Perceptual loss using VGG16
# -------------------------------
class VGGPerceptualLoss(nn.Module):
    def __init__(self, layer_ids=[3, 8, 15], device='cuda'):
        super().__init__()
        self.device = device
        vgg = models.vgg16(pretrained=True).features.to(device)  # <-- move to device
        self.layers = nn.ModuleList([vgg[:i+1] for i in layer_ids])
        for param in vgg.parameters():
            param.requires_grad = False

    def forward(self, x, y):
        # normalize to ImageNet stats
        mean = torch.tensor([0.485, 0.456, 0.406], device=self.device).view(1,3,1,1)
        std  = torch.tensor([0.229, 0.224, 0.225], device=self.device).view(1,3,1,1)
        x_norm = (x - mean) / std
        y_norm = (y - mean) / std

        loss = 0
        for layer in self.layers:
            loss += F.mse_loss(layer(x_norm), layer(y_norm))
        return loss

# -------------------------------
# VAE Loss combining L1 + Perceptual + KL
# -------------------------------
class VAEVggLoss(nn.Module):
    def __init__(self, recon_weight=1.0, perc_weight=0.1, kl_weight=0.01):
        super().__init__()
        self.recon_weight = recon_weight
        self.perc_weight = perc_weight
        self.kl_weight = kl_weight
        self.perc_loss = VGGPerceptualLoss()

    def forward(self, x_recon, x, mu, logvar):
        # L1 reconstruction loss
        recon_loss = F.l1_loss(x_recon, x)

        # Perceptual loss
        perc_loss = self.perc_loss(x_recon, x)

        # KL divergence
        kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        kl_loss = kl_loss / x.size(0)  # normalize by batch size

        total_loss = self.recon_weight * recon_loss + \
                     self.perc_weight * perc_loss + \
                     self.kl_weight * kl_loss
        return total_loss, recon_loss, perc_loss, kl_loss
vgg_loss = VAEVggLoss()

In [ ]:
epochs = 200
if train_mode:
    for epoch in range(current_epoch + 1, current_epoch + epochs + 1):
        model.train()
        running_total = running_recon = running_kld = 0.0

        for batch_idx, (x, _) in enumerate(tqdm(train_loader), start=1):
            x = x.to(device)
            optimizer.zero_grad(set_to_none=True)
            logits, mean, logvar = model(x)
            loss, l1_loss, perc_loss, kl_loss = vgg_loss(logits, x, mean, logvar)
            # loss, l1_loss, perc_loss, kl_loss = criterion(x_recon, x, mu, logvar)
            loss.backward()
            optimizer.step()
            if batch_idx % 3000 == 0:
                print(f"Epoch [{epoch+1}/{epochs}], Step [{epoch}/{len(train_loader)}], "
                  f"Total Loss: {loss.item():.4f}, "
                  f"L1: {l1_loss.item():.4f}, "
                  f"Perceptual: {perc_loss.item():.4f}, "
                  f"KL: {kl_loss.item():.4f}")

            # running_total += loss.total.item()
            # running_recon += loss.recon.item()
            # running_kld += loss.kld.item()
        current_epoch += 1
        # beta = min(0.8, beta * beta_anneal_factor)

        save_reconstructions(model, x, out_dir, current_epoch, device)
        n_train = len(train_loader.dataset)

        # print(
        #      f"Epoch {epoch:02d} | loss: {loss.item}
        # print(
        #     f"Epoch {epoch:02d} | total: {running_total / n_train:.4f} | "
        #     f"recon: {running_recon / n_train:.4f} | kld: {running_kld / n_train:.4f} | "
        #     f"beta: {beta:.4f}"
        # )
        # if(epoch % 10) == 0:
        #     training_loss_tracker.append([running_total / n_train, running_recon / n_train, running_kld / n_train, beta])

        if epoch % 100 == 0:
            save_checkpoint(model, optimizer, epoch, checkpoint_dir)

 95%|█████████▍| 3002/3166 [06:25<00:20,  8.11it/s]

Epoch [48/200], Step [47/3166], Total Loss: 3.7064, L1: 0.0987, Perceptual: 27.6040, KL: 84.7289


 95%|█████████▍| 3002/3166 [06:39<00:20,  8.09it/s]

Epoch [49/200], Step [48/3166], Total Loss: 3.5058, L1: 0.1069, Perceptual: 27.5072, KL: 64.8140


 95%|█████████▍| 3002/3166 [06:41<00:20,  8.07it/s]

...

 95%|█████████▍| 3002/3166 [07:08<00:21,  7.50it/s]

Epoch [106/200], Step [105/3166], Total Loss: 2.6828, L1: 0.1035, Perceptual: 22.0212, KL: 37.7165


 95%|█████████▍| 3002/3166 [07:06<00:21,  7.55it/s]

Epoch [107/200], Step [106/3166], Total Loss: 2.7350, L1: 0.1080, Perceptual: 22.5878, KL: 36.8230


 42%|████▏     | 1320/3166 [03:12<04:30,  6.82it/s]